# Importaciones y carga inicial de datos

In [1]:
from datetime import datetime
import pandas as pd
from pandas.tseries.offsets import MonthEnd
from dateutil.relativedelta import relativedelta
import calendar

# Carga archivos Excel
data = pd.read_excel(r'D:\Datos\Documents\GitHub\2025\Agente_simuldor_libranza\Agente_simuldor_libranza\1.Datos\2.Proccesed\Data_final.xlsx')
data_pagaduria = pd.read_excel(r'D:\Datos\Documents\GitHub\2025\Agente_simuldor_libranza\Agente_simuldor_libranza\1.Datos\2.Proccesed\Fecha_pagaduria.xlsx')


# Parametros y variables base para la simulacion

In [2]:
# Número de libranza a consultar
nro_libranza = 55263

# Obtener fecha actual
hoy = datetime.today()

# Obtener pagaduría según libranza
pagaduria = data.loc[data["nolibra"] == nro_libranza, "pagaduria"].values[0]

# Asegurar columna FECHA en formato datetime
data_pagaduria["FECHA"] = pd.to_datetime(data_pagaduria["FECHA"])

# Obtener fecha programada de pagaduría
fecha_programada = pd.to_datetime(data_pagaduria.loc[data_pagaduria["PAGADURIA"] == pagaduria, "FECHA"].values[0])


# Calcular corte_saldo_sistema
if hoy.date() > fecha_programada.date():
    año, mes = hoy.year, hoy.month
else:
    año, mes = (hoy.year - 1, 12) if hoy.month == 1 else (hoy.year, hoy.month - 1)

ultimo_dia = calendar.monthrange(año, mes)[1]
corte_saldo_sistema = datetime(año, mes, ultimo_dia)

print("Corte saldo sistema:", corte_saldo_sistema.strftime("%Y-%m-%d"))


Corte saldo sistema: 2025-08-31


# Obtener datos del prestamos desde el DataFrame

In [3]:
# Monto total del préstamo
monto_prestado = data.loc[data["nolibra"] == nro_libranza, "BASE CAPITAL"].values[0]

# Tasa mensual nominal (decimal)
tasa_mensual = data.loc[data["nolibra"] == nro_libranza, "TASA CRÉDITO"].values[0]

# Plazo en meses
plazo_meses = data.loc[data["nolibra"] == nro_libranza, "Plazo"].values[0]
plazo = plazo_meses  # para usar un nombre corto

# Seguro deudor mensual
seguro_deudor = data.loc[data["nolibra"] == nro_libranza, "SEGURO DEUDORES"].values[0] / plazo_meses

# Fecha inicio crédito (datetime)
fecha1 = pd.to_datetime(data.loc[data["nolibra"] == nro_libranza, "feinicre2"].values[0])
fecha2 = pd.to_datetime(data.loc[data["nolibra"] == nro_libranza, "Fecha_primer_pago"].values[0])
fecha_inicio = min(fecha1, fecha2)

# Saldo cierre mes 0
saldo_cierre_mes_0 = data.loc[data["nolibra"] == nro_libranza, "saldo cierre MAYO 2025 0"].values[0]

# Cuotas pagas mes
cuotas_paga_mes = data.loc[data["nolibra"] == nro_libranza, "cuotas Pagas"].values[0]

# Cuotas causadas calculadas
diff = relativedelta(corte_saldo_sistema, pd.to_datetime(data.loc[data["nolibra"] == nro_libranza, "feinicre2"].values[0]))
cuotas_causadas = diff.years * 12 + diff.months + 1

# Cuota definida
cuotadef = data.loc[data["nolibra"] == nro_libranza, "cuotadef"].values[0]

print(monto_prestado, tasa_mensual, seguro_deudor, fecha_inicio, saldo_cierre_mes_0, cuotas_paga_mes, cuotas_causadas, cuotadef, plazo)


2559500.0 0.0212 5500.0 2022-07-31 00:00:00 6754644 36.0 38 62543 144


In [4]:
print(fecha_inicio)

2022-07-31 00:00:00


# Calculo de cuotas, amortizacion y generacion de fechas de pago 

In [5]:
# Fechas de pago (al final de cada mes)
fechas_pago = pd.date_range(start=fecha_inicio, periods=plazo, freq="M")

# Cuota fija mensual (fórmula francesa)
cuota = monto_prestado * (tasa_mensual * (1 + tasa_mensual) ** plazo) / ((1 + tasa_mensual) ** plazo - 1)

# Saldo inicial
saldo = monto_prestado

# Inicializar tabla amortización con fila saldo inicial
tabla = [{
    "N°": 0,
    "FECHA": "",
    "TASA LIQUIDACION": "",
    "CUOTA": "",
    "SEGURO DEUDORES": round(seguro_deudor),
    "INTERES": "",
    "AMORTIZACION": "",
    "SALDO": round(saldo)
}]


C:\Users\cientifico.datos\AppData\Local\Temp\ipykernel_14968\2090957796.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  fechas_pago = pd.date_range(start=fecha_inicio, periods=plazo, freq="M")


# Loop mensual para clcular cuotas, interes, amortizacion y saldo

In [6]:
for i in range(int(plazo_meses)):
    interes = saldo * tasa_mensual
    amortizacion = cuota - interes

    if i == plazo_meses - 1:
        amortizacion = saldo
        cuota = interes + amortizacion

    saldo -= amortizacion

    fila = {
        "N°": i + 1,
        "FECHA": fechas_pago[i].strftime("%d/%m/%Y"),
        "TASA LIQUIDACION": f"{tasa_mensual * 100:.5f}%",
        "CUOTA": round(cuota),
        "SEGURO DEUDORES": round(seguro_deudor),
        "INTERES": round(interes),
        "AMORTIZACION": round(amortizacion),
        "SALDO": round(max(saldo, 0))
    }
    tabla.append(fila)

# Convertir a DataFrame
df = pd.DataFrame(tabla)

# Convertir FECHA a datetime para merge posterior
df["FECHA"] = pd.to_datetime(df["FECHA"], format="%d/%m/%Y", errors="coerce")


# Carga tasa de usura y crear rango de fechas

In [7]:
df_tasa_usura = pd.read_excel(r'D:\Datos\Documents\GitHub\2025\Agente_simuldor_libranza\Agente_simuldor_libranza\1.Datos\2.Proccesed\Tasa_usura.xlsx')
df_tasa_usura["Fecha Inicio"] = pd.to_datetime(df_tasa_usura["Fecha Inicio"])
df_tasa_usura["Fecha Fin"] = pd.to_datetime(df_tasa_usura["Fecha Fin"])

# Expandir rangos de fechas para usura (un día por fila)
rangos_usura = []
for _, fila in df_tasa_usura.iterrows():
    fechas = pd.date_range(start=fila["Fecha Inicio"], end=fila["Fecha Fin"], freq="D")
    for fecha in fechas:
        rangos_usura.append({"FECHA": fecha, "TASA_USURA": fila["TASA_USURA"]})

df_rangos_usura = pd.DataFrame(rangos_usura)


# Unir con tasa de usura y reordenar columnas

In [8]:
df = df.merge(df_rangos_usura, on="FECHA", how="left")
df["TASA_USURA"] = df["TASA_USURA"].fillna(0)

# Reordenar para que tasa_usura quede justo después de tasa_liquidacion
columnas = df.columns.tolist()
posicion = columnas.index("TASA LIQUIDACION") + 1
columnas.insert(posicion, columnas.pop(columnas.index("TASA_USURA")))
df = df[columnas]


# Armar nueva taza de amortizacion

In [9]:
df_ajustado = df.copy()

# Convertir tasa_liquidacion a decimal
df_ajustado["TASA_LIQUIDACION_VALOR"] = (
    pd.to_numeric(
        df_ajustado["TASA LIQUIDACION"].str.replace("%", "", regex=False).str.replace(",", "."),
        errors="coerce"
    ) / 100
)

# Calcular tasa aplicada (mínimo entre tasa_liq y tasa_usura si existe)
df_ajustado["TASA APLICADA"] = df_ajustado.apply(
    lambda fila: fila["TASA_LIQUIDACION_VALOR"]
    if pd.isna(fila.get("TASA_USURA", float("nan"))) or fila.get("TASA_USURA", 0) == 0
    else min(fila["TASA_LIQUIDACION_VALOR"], fila["TASA_USURA"]),
    axis=1
)

# Obtener cuota de fila 1 para usar después
cuota = df_ajustado.loc[1, "CUOTA"] if 1 in df_ajustado.index and "CUOTA" in df_ajustado.columns else 0

# Variables iniciales
saldo = monto_prestado
acumulado_interes = 0

def formatear(numero, decimales=0):
    if pd.isna(numero):
        return ""
    return f"{numero:,.{decimales}f}".replace(",", "X").replace(".", ",").replace("X", ".")

filas = []
filas.append({
    "Nº": 0,
    "FECHA": "",
    "TASA_LIQUIDACIÓN": "",
    "TASA DE USURA": "",
    "SEGURO DEUDORES": "",
    "FINANCIACIÓN CAPITAL": "",
    "AMORTIZACIÓN CAPITAL": "",
    "SALDO CAPITAL": formatear(saldo, 0),
    "Acumulado_financiacion_capital": ""
})

# Paso 7: recorrer el plazo completo
for i in range(1, round(plazo) + 1):
    f = df_ajustado.iloc[i]

    tasa_liq = f["TASA_LIQUIDACION_VALOR"]
    tasa_usura = f.get("TASA_USURA", float("nan"))
    tasa_aplicada = f["TASA APLICADA"]

    interes = saldo * tasa_aplicada
    amort = cuota - interes
    saldo -= amort
    acumulado_interes += interes

    # Manejo de fecha
    fecha_str = ""
    if "FECHA" in f and pd.notna(f["FECHA"]):
        try:
            fecha_str = pd.to_datetime(f["FECHA"]).strftime("%d/%m/%Y")
        except Exception:
            fecha_str = str(f["FECHA"])

    filas.append({
        "Nº": int(f["N°"]) if "N°" in f and pd.notna(f["N°"]) else i,
        "FECHA": fecha_str,
        "TASA_LIQUIDACIÓN": formatear(tasa_liq * 100, 5) + "%" if pd.notna(tasa_liq) else "",
        "TASA DE USURA": "" if pd.isna(tasa_usura) or tasa_usura == 0 else formatear(tasa_usura * 100, 6) + "%",
        "SEGURO DEUDORES": formatear(f.get("SEGURO DEUDORES", 0), 0),
        "FINANCIACIÓN CAPITAL": formatear(interes, 0),
        "AMORTIZACIÓN CAPITAL": formatear(amort, 0),
        "SALDO CAPITAL": formatear(saldo, 0),
        "Acumulado_financiacion_capital": formatear(acumulado_interes, 0)
    })

# Paso 8: construir el DataFrame final
df_final = pd.DataFrame(filas)[[
    "Nº", "FECHA", "TASA_LIQUIDACIÓN", "TASA DE USURA",
    "SEGURO DEUDORES", "FINANCIACIÓN CAPITAL",
    "AMORTIZACIÓN CAPITAL", "SALDO CAPITAL", "Acumulado_financiacion_capital"
]]

# Resultado listo para mostrar o exportar
df_final.head(40)

,Nº,FECHA,TASA_LIQUIDACIÓN,TASA DE USURA,SEGURO DEUDORES,FINANCIACIÓN CAPITAL,AMORTIZACIÓN CAPITAL,SALDO CAPITAL,Acumulado_financiacion_capital
0,0,,,,,,,2.559.500,
1,1,31/07/2022,"2,12000%","2,335399%",5.500,54.261,2.782,2.556.718,54.261
2,2,31/08/2022,"2,12000%","2,425144%",5.500,54.202,2.841,2.553.878,108.464
3,3,30/09/2022,"2,12000%","2,548215%",5.500,54.142,2.901,2.550.977,162.606
4,4,31/10/2022,"2,12000%","2,653141%",5.500,54.081,2.962,2.548.015,216.687
5,5,30/11/2022,"2,12000%","2,761841%",5.500,54.018,3.025,2.544.990,270.705
6,6,31/12/2022,"2,12000%","2,932567%",5.500,53.954,3.089,2.541.900,324.658
7,7,31/01/2023,"2,12000%","3,041082%",5.500,53.888,3.155,2.538.746,378.547
8,8,28/02/2023,"2,12000%","3,160790%",5.500,53.821,3.222,2.535.524,432.368
9,9,31/03/2023,"2,12000%","3,219194%",5.500,53.753,3.290,2.532.234,486.121


# Calculo saldo actual

In [10]:
from dateutil.relativedelta import relativedelta

### CALCULAR CUOTAS PAGAS ###

# Asegúrate que fecha_data es un datetime (no una serie)
fecha_data = pd.to_datetime(data.loc[data["nolibra"] == nro_libranza, "FECHA_DATA"].values[0])

# Calcular diferencia en meses entre fecha_data y corte
diff = relativedelta(corte_saldo_sistema, fecha_data)
meses_adicionales = diff.years * 12 + diff.months

# Cuotas pagadas = las que ya se habían pagado a la fecha_data + las adicionales desde entonces
cuotas_pagas_base = plazo - (saldo_cierre_mes_0 / cuotadef)
cuotas_pagas = cuotas_pagas_base + meses_adicionales

print(f"Cuotas Pagas: {cuotas_pagas:.0f}")
print(f"Cuotas Causadas: {cuotas_causadas:.0f}")


Cuotas Pagas: 38
Cuotas Causadas: 38


In [11]:
print(plazo)

144


In [ ]:
print(tasa_usura_str)

In [ ]:
df_final.head(40)

,Nº,FECHA,TASA_LIQUIDACIÓN,TASA DE USURA,SEGURO DEUDORES,FINANCIACIÓN CAPITAL,AMORTIZACIÓN CAPITAL,SALDO CAPITAL,Acumulado_financiacion_capital
0,0,,,,,,,2.559.500,
1,1,31/07/2022,"2,12000%","2,335399%",5.500,54.261,2.782,2.556.718,54.261
2,2,31/08/2022,"2,12000%","2,425144%",5.500,54.202,2.841,2.553.878,108.464
3,3,30/09/2022,"2,12000%","2,548215%",5.500,54.142,2.901,2.550.977,162.606
4,4,31/10/2022,"2,12000%","2,653141%",5.500,54.081,2.962,2.548.015,216.687
5,5,30/11/2022,"2,12000%","2,761841%",5.500,54.018,3.025,2.544.990,270.705
6,6,31/12/2022,"2,12000%","2,932567%",5.500,53.954,3.089,2.541.900,324.658
7,7,31/01/2023,"2,12000%","3,041082%",5.500,53.888,3.155,2.538.746,378.547
8,8,28/02/2023,"2,12000%","3,160790%",5.500,53.821,3.222,2.535.524,432.368
9,9,31/03/2023,"2,12000%","3,219194%",5.500,53.753,3.290,2.532.234,486.121


In [13]:
import calendar
from datetime import datetime

# Paso 2: Obtener tasas en formato string desde df_final
tasa_usura_str = df_final.loc[df_final["Nº"] == cuotas_pagas, "TASA DE USURA"].values[0]
tasa_liq_str = df_final.loc[df_final["Nº"] == cuotas_pagas, "TASA_LIQUIDACIÓN"].values[0]

# Paso 3: Limpiar y convertir a float
tasa_usura = float(tasa_usura_str.replace('%', '').replace(',', '.'))
tasa_liq = float(tasa_liq_str.replace('%', '').replace(',', '.'))

# Paso 4: Elegir tasa mínima y pasar a decimal
tasa = min(tasa_usura, tasa_liq) / 100
print(f"Tasa aplicada: {tasa:.6f}")

# Paso 5: Extraer saldo capital y convertir a float
saldo_capital_str = df_final.loc[df_final["Nº"] == cuotas_pagas, "SALDO CAPITAL"].values[0]
saldo_capital = float(str(saldo_capital_str).replace('.', '').replace(',', '.'))
print(f"Saldo capital base: {saldo_capital}")

# Paso 6: Calcular subtotal obligación mensual por intereses
dias_mes = calendar.monthrange(datetime.today().year, datetime.today().month)[1]
subtotal_obligacion_mensual = (saldo_capital * tasa) / dias_mes

# Paso 7: Calcular días restantes del mes y subtotal obligación proporcional
dias_restantes = dias_mes - datetime.today().day
subtotal_obligacion_proporcional = subtotal_obligacion_mensual * dias_restantes

print(f"Días restantes mes: {dias_restantes}")
print(f"Subtotal obligación proporcional (intereses prorrateados): {subtotal_obligacion_proporcional:.2f}")

# Función para extraer acumulados limpiando formato
def obtener_acumulado(df, numero):
    valor = df.loc[df["Nº"] == numero, "Acumulado_financiacion_capital"].values
    if len(valor) == 0 or str(valor[0]).strip() == '':
        return 0.0
    valor_limpio = str(valor[0]).replace('.', '').replace(',', '.')
    return float(valor_limpio)

# Paso 8: Comparar cuotas pagas y causadas para calcular ajuste acumulado
if cuotas_pagas == cuotas_causadas:
    resultado = 0
elif cuotas_causadas > cuotas_pagas:
    resultado = obtener_acumulado(df_final, cuotas_causadas) - obtener_acumulado(df_final, cuotas_pagas)
else:
    resultado = obtener_acumulado(df_final, cuotas_pagas) - obtener_acumulado(df_final, cuotas_causadas)
print(f"Diferencia acumulados (resultado): {resultado:.2f}")

# Paso 9: Calcular base cálculo para días de liquidación (saldo capital a cuotas causadas)
fila_base_calculo = cuotas_causadas
valor_base_calc = df_final.loc[df_final["Nº"] == fila_base_calculo, "SALDO CAPITAL"].values
base_calculo = float(str(valor_base_calc[0]).replace('.', '').replace(',', '.')) if len(valor_base_calc) else 0.0

# Paso 10: Calcular días para liquidar y resultado proporcional
fecha_hoy = datetime.today()
dias_liquidar = (fecha_hoy - corte_saldo_sistema).days + 1
if fecha_hoy == corte_saldo_sistema:
    resultado_2 = 0
else:
    resultado_2 = base_calculo * (tasa / 30) * dias_liquidar
print(f"Resultado días liquidar (resultado_2): {resultado_2:.2f}")

# Paso 11: Calcular saldo de seguros proporcional
saldo_seguros = (cuotas_causadas * seguro_deudor) - (cuotas_pagas * seguro_deudor)
print(f"Saldo seguros: {saldo_seguros:.2f}")

# Paso 12: Calcular saldo actual sumando todos los componentes
saldo_actual = saldo_capital - resultado + resultado_2 + saldo_seguros
print(f"Saldo actual calculado: {saldo_actual:.2f}")

ValueError: could not convert string to float: ''